# Generative AI for Querying Databases

Processed data saved in a database table can be accessed, based on your requirements, using queries. Because queries are an essential part of any data professional's workflow, writing efficient queries is a necessary skillset. In this lab, you will learn how you can leverage Generative AI platforms to create optimized queries for your data, provided you can give the model enough context.

For the purpose of this lab, you are making use of the Heart Disease Data set from the UCI ML library, available publically under the CCA 4.0 International license.

You can download the data set and run the queries generated in this lab using any SQL querying system.

## Giving the context


If you provide the model with the description of your data, you can generate efficient and readily usable queries for fetching the data based on your requirements.

Paste the following text in the prompt instructions to give the model the appropriate context for the data

We have a Heart Disease prediction dataset with a single table which has the following attributes:

1. **age** - age in years
2. **gender**- gender (1 = male; 0 = female)
3. **cp** - chest pain type:

        -- Value 1: typical angina
        -- Value 2: atypical angina
        -- Value 3: non-anginal pain
        -- Value 4: asymptomatic

4. **trestbps** - resting blood pressure (in mm Hg on admission to the hospital)
5. **chol** - serum cholestoral in mg/dl
6. **fbs** - (fasting blood sugar > 120 mg/dl)  (1 = true; 0 = false)
7. **restecg** - resting electrocardiographic results:

        -- Value 0: normal
        -- Value 1: having ST-T wave abnormality (T wave inversions and/or ST elevation or depression of > 0.05 mV)
        -- Value 2: showing probable or definite left ventricular hypertrophy by Estes' criteria

8. **thalach** - maximum heart rate achieved
9. **exang** - exercise induced angina (1 = yes; 0 = no)
10. **oldpeak** - ST depression induced by exercise relative to rest
11. **slope** - the slope of the peak exercise ST segment:

        -- Value 1: upsloping
        -- Value 2: flat
        -- Value 3: downsloping

12. **ca** - number of major vessels (0-3) colored by flourosopy
13. **thal** - 3 = normal; 6 = fixed defect; 7 = reversable defect
14. **num** (the predicted attribute) - diagnosis of heart disease (angiographic disease status):

        -- Value 0: < 50% diameter narrowing
        -- Value 1: > 50% diameter narrowing

### Creating DB connection 

In [2]:
%load_ext sql
%sql sqlite:///UCI_database.db

In [3]:
import csv, sqlite3
import prettytable
import pandas as pd
prettytable.DEFAULT = 'DEFAULT'

con = sqlite3.connect("UCI_database.db")
cur = con.cursor()

### Fetching the data from [UCI ML Repository](https://archive.ics.uci.edu/dataset/45/heart+disease)

In [4]:
!pip install ucimlrepo


[notice] A new release of pip is available: 25.2 -> 25.3
[notice] To update, run: python.exe -m pip install --upgrade pip


In [5]:
from ucimlrepo import fetch_ucirepo 
  
# fetch dataset 
heart_disease = fetch_ucirepo(id=45) 
  
# data (as pandas dataframes) 
X = heart_disease.data.features 
y = heart_disease.data.targets 
  
# metadata 
print(heart_disease.metadata) 
  
# variable information 
print(heart_disease.variables) 


{'uci_id': 45, 'name': 'Heart Disease', 'repository_url': 'https://archive.ics.uci.edu/dataset/45/heart+disease', 'data_url': 'https://archive.ics.uci.edu/static/public/45/data.csv', 'abstract': '4 databases: Cleveland, Hungary, Switzerland, and the VA Long Beach', 'area': 'Health and Medicine', 'tasks': ['Classification'], 'characteristics': ['Multivariate'], 'num_instances': 303, 'num_features': 13, 'feature_types': ['Categorical', 'Integer', 'Real'], 'demographics': ['Age', 'Sex'], 'target_col': ['num'], 'index_col': None, 'has_missing_values': 'yes', 'missing_values_symbol': 'NaN', 'year_of_dataset_creation': 1989, 'last_updated': 'Fri Nov 03 2023', 'dataset_doi': '10.24432/C52P4X', 'creators': ['Andras Janosi', 'William Steinbrunn', 'Matthias Pfisterer', 'Robert Detrano'], 'intro_paper': {'ID': 231, 'type': 'NATIVE', 'title': 'International application of a new probability algorithm for the diagnosis of coronary artery disease.', 'authors': 'R. Detrano, A. Jánosi, W. Steinbrunn, M

In [6]:
df=pd.read_csv('https://archive.ics.uci.edu/static/public/45/data.csv')
df.head()

,age,sex,cp,trestbps,chol,fbs,restecg,thalach,exang,oldpeak,slope,ca,thal,num
0,63,1,1,145,233,1,2,150,0,2.3,3,0.0,6.0,0
1,67,1,4,160,286,0,2,108,1,1.5,2,3.0,3.0,2
2,67,1,4,120,229,0,2,129,1,2.6,2,2.0,7.0,1
3,37,1,3,130,250,0,0,187,0,3.5,3,0.0,3.0,0
4,41,0,2,130,204,0,2,172,0,1.4,1,0.0,3.0,0


In [7]:
df.shape

(303, 14)

Creating DB Table from Pandas Dataframe

In [8]:
df.to_sql('HEART_DISEASE_TABLE', con, if_exists='replace', index=False)

303

### PROMPT EXCERCISES

**PROMPT:** "Write an SQL query to find the minimum, maximum, and average age of patients in the dataset."

In [9]:
%%sql
SELECT 
    MIN(age) AS min_age,
    MAX(age) AS max_age,
    AVG(age) AS avg_age
FROM HEART_DISEASE_TABLE;

 * sqlite:///UCI_database.db
Done.


min_age,max_age,avg_age
29,77,54.43894389438944


**PROMPT:** "Write and SQL query to count the number of male and female patients in the dataset."

In [13]:
%%sql
-- Option A: count by raw gender values (0 = female, 1 = male)
SELECT
  sex,
  COUNT(*) AS count
FROM HEART_DISEASE_TABLE
GROUP BY sex
ORDER BY sex;



 * sqlite:///UCI_database.db
Done.


sex,count
0,97
1,206


In [17]:
%%sql
-- Option B: count by human-readable labels for clarity
SELECT
  CASE
    WHEN sex = 1 THEN 'Male'
    WHEN sex = 0 THEN 'Female'
    ELSE 'Unknown'
  END AS gender_label,
  COUNT(*) AS count
FROM HEART_DISEASE_TABLE
GROUP BY
  CASE
    WHEN sex = 1 THEN 'Male'
    WHEN sex = 0 THEN 'Female'
    ELSE 'Unknown'
  END
ORDER BY gender_label;

 * sqlite:///UCI_database.db
Done.


gender_label,count
Female,97
Male,206


**PROMPT:** "Write an SQL query to determine the frequency of each type of chest pain (typical angina, atypical angina, non-anginal pain, asymptomatic) among patients."

In [18]:
%%sql
-- Frequency of each chest pain type (cp: 1=typical, 2=atypical, 3=non-anginal, 4=asymptomatic)
SELECT
  CASE cp
    WHEN 1 THEN 'Typical angina'
    WHEN 2 THEN 'Atypical angina'
    WHEN 3 THEN 'Non-anginal pain'
    WHEN 4 THEN 'Asymptomatic'
    ELSE 'Unknown'
  END AS chest_pain_type,
  COUNT(*) AS frequency
FROM HEART_DISEASE_TABLE
WHERE cp IS NOT NULL
  AND cp IN (1, 2, 3, 4)
GROUP BY
  CASE cp
    WHEN 1 THEN 'Typical angina'
    WHEN 2 THEN 'Atypical angina'
    WHEN 3 THEN 'Non-anginal pain'
    WHEN 4 THEN 'Asymptomatic'
  END
ORDER BY frequency DESC;

 * sqlite:///UCI_database.db
Done.


chest_pain_type,frequency
Asymptomatic,144
Non-anginal pain,86
Atypical angina,50
Typical angina,23


**PROMPT:** "Write an SQL query to investigate the distribution of the target variable (presence or absence of heart disease) within different age groups (e.g., 20-30, 30-40, etc.)."

In [19]:
%%sql
SELECT
  CASE
    WHEN age IS NULL THEN 'Unknown'
    WHEN age >= 20 AND age < 30 THEN '20-29'
    WHEN age >= 30 AND age < 40 THEN '30-39'
    WHEN age >= 40 AND age < 50 THEN '40-49'
    WHEN age >= 50 AND age < 60 THEN '50-59'
    WHEN age >= 60 AND age < 70 THEN '60-69'
    WHEN age >= 70 AND age < 80 THEN '70-79'
    ELSE '80+'
  END AS age_group,
  num AS heart_disease_status,
  COUNT(*) AS count
FROM HEART_DISEASE_TABLE
GROUP BY
  CASE
    WHEN age IS NULL THEN 'Unknown'
    WHEN age >= 20 AND age < 30 THEN '20-29'
    WHEN age >= 30 AND age < 40 THEN '30-39'
    WHEN age >= 40 AND age < 50 THEN '40-49'
    WHEN age >= 50 AND age < 60 THEN '50-59'
    WHEN age >= 60 AND age < 70 THEN '60-69'
    WHEN age >= 70 AND age < 80 THEN '70-79'
    ELSE '80+'
  END,
  num
ORDER BY age_group, heart_disease_status;

 * sqlite:///UCI_database.db
Done.


age_group,heart_disease_status,count
20-29,0,1
30-39,0,10
30-39,1,2
30-39,3,1
30-39,4,1
40-49,0,50
40-49,1,11
40-49,2,5
40-49,3,6
50-59,0,65


## Practice Prompts

1. Cholesterol Range:

- **PROMPT:** "Find the range of cholesterol levels among patients (minimum, maximum)."

In [20]:
%%sql 
SELECT
  MIN(chol) AS min_chol,
  MAX(chol) AS max_chol
FROM HEART_DISEASE_TABLE;

 * sqlite:///UCI_database.db
Done.


min_chol,max_chol
126,564


2. Age Range and Gender Analysis:

- **PROMPT:** "Determine the age range (youngest and oldest) for male and female patients separately."

In [21]:
%%sql  
-- Option A: Age range by raw gender value (0 = Female, 1 = Male)
SELECT
  sex,
  MIN(age) AS min_age,
  MAX(age) AS max_age
FROM HEART_DISEASE_TABLE
WHERE age IS NOT NULL
GROUP BY sex
ORDER BY sex;


 * sqlite:///UCI_database.db
Done.


sex,min_age,max_age
0,34,76
1,29,77


In [22]:
%%sql
-- Option B: Age range by human-readable gender label
SELECT
  CASE
    WHEN sex = 1 THEN 'Male'
    WHEN sex = 0 THEN 'Female'
    ELSE 'Unknown'
  END AS gender_label,
  MIN(age) AS min_age,
  MAX(age) AS max_age
FROM HEART_DISEASE_TABLE
WHERE age IS NOT NULL
GROUP BY
  CASE
    WHEN sex = 1 THEN 'Male'
    WHEN sex = 0 THEN 'Female'
    ELSE 'Unknown'
  END
ORDER BY gender_label;

 * sqlite:///UCI_database.db
Done.


gender_label,min_age,max_age
Female,34,76
Male,29,77


3. Age Group Analysis and Target Variable:

- **PROMPT:** "Investigate the distribution of the target variable (presence or absence of heart disease) within different age groups (e.g., 20-30, 30-40, etc.)."


In [23]:
%%sql 
SELECT
  age_group,
  SUM(CASE WHEN num = 1 THEN 1 ELSE 0 END) AS disease_count,
  SUM(CASE WHEN num = 0 THEN 1 ELSE 0 END) AS non_disease_count,
  COUNT(*) AS total_in_group,
  ROUND(SUM(CASE WHEN num = 1 THEN 1 ELSE 0 END) * 1.0 / NULLIF(COUNT(*), 0), 4) AS disease_rate
FROM (
  SELECT
    CASE
      WHEN age >= 20 AND age < 30 THEN '20-29'
      WHEN age >= 30 AND age < 40 THEN '30-39'
      WHEN age >= 40 AND age < 50 THEN '40-49'
      WHEN age >= 50 AND age < 60 THEN '50-59'
      WHEN age >= 60 AND age < 70 THEN '60-69'
      WHEN age >= 70 AND age < 80 THEN '70-79'
      WHEN age >= 80 THEN '80+'
      ELSE 'Unknown'
    END AS age_group,
    num
  FROM HEART_DISEASE_TABLE
  WHERE age IS NOT NULL
) AS t
GROUP BY age_group
ORDER BY
  CASE age_group
    WHEN '20-29' THEN 1
    WHEN '30-39' THEN 2
    WHEN '40-49' THEN 3
    WHEN '50-59' THEN 4
    WHEN '60-69' THEN 5
    WHEN '70-79' THEN 6
    WHEN '80+' THEN 7
    ELSE 8
  END;

 * sqlite:///UCI_database.db
Done.


age_group,disease_count,non_disease_count,total_in_group,disease_rate
20-29,0,1,1,0.0
30-39,2,10,14,0.1429
40-49,11,50,72,0.1528
50-59,24,65,125,0.192
60-69,17,32,81,0.2099
70-79,1,6,10,0.1


4. Maximum Heart Rate by Age Group:

- **PROMPT:** "Find the maximum heart rate achieved during exercise for different age groups (e.g., 30-40, 40-50, etc.)."

In [24]:
%%sql 
-- Find the maximum heart rate achieved (thalach) for different 10-year age groups.
-- Age bins: 20-29, 30-39, 40-49, 50-59, 60-69, 70-79, 80+.
-- Replace 'HEART_DISEASE_TABLE' with the actual table name.

SELECT
  age_group,
  MAX(thalach) AS max_thalach
FROM (
  SELECT
    CASE
      WHEN age >= 20 AND age < 30 THEN '20-29'
      WHEN age >= 30 AND age < 40 THEN '30-39'
      WHEN age >= 40 AND age < 50 THEN '40-49'
      WHEN age >= 50 AND age < 60 THEN '50-59'
      WHEN age >= 60 AND age < 70 THEN '60-69'
      WHEN age >= 70 AND age < 80 THEN '70-79'
      WHEN age >= 80 THEN '80+'
      ELSE 'Unknown'
    END AS age_group,
    thalach
  FROM HEART_DISEASE_TABLE
  WHERE age IS NOT NULL AND thalach IS NOT NULL
) AS sub
GROUP BY age_group
ORDER BY
  CASE age_group
    WHEN '20-29' THEN 1
    WHEN '30-39' THEN 2
    WHEN '40-49' THEN 3
    WHEN '50-59' THEN 4
    WHEN '60-69' THEN 5
    WHEN '70-79' THEN 6
    WHEN '80+' THEN 7
    ELSE 8
  END;

 * sqlite:///UCI_database.db
Done.


age_group,max_thalach
20-29,202
30-39,192
40-49,194
50-59,195
60-69,179
70-79,162


5. Percentage of Patients with High Blood Sugar:

- **PROMPT:** "Calculate the percentage of patients with fasting blood sugar greater than 120 mg/dl."

In [25]:
%%sql 
-- Percentage of patients with fasting blood sugar > 120 mg/dL
-- fbs: 1 = true, 0 = false. Calculate over non-null values only.
SELECT
  ROUND(AVG(fbs) * 100.0, 2) AS pct_fbs_gt_120
FROM HEART_DISEASE_TABLE
WHERE fbs IS NOT NULL;

 * sqlite:///UCI_database.db
Done.


pct_fbs_gt_120
14.85


6. Ratio of Patients with Resting Electrocardiographic Abnormality:

- **PROMPT:** "Find the ratio of patients with abnormal resting electrocardiographic results to those with normal results."

In [26]:
%%sql 
-- Ratio of patients with abnormal resting ECG results to those with normal results
-- Abnormal is defined as restecg <> 0 (i.e., 1 or 2). Normal is restecg = 0.
-- NULL restecg values are ignored.
WITH counts AS (
  SELECT
    SUM(CASE WHEN restecg = 0 THEN 1 ELSE 0 END) AS normal_count,
    SUM(CASE WHEN restecg <> 0 THEN 1 ELSE 0 END) AS abnormal_count
  FROM HEART_DISEASE_TABLE
  WHERE restecg IS NOT NULL
)
SELECT
  abnormal_count,
  normal_count,
  CASE
    WHEN normal_count = 0 THEN NULL
    ELSE abnormal_count * 1.0 / normal_count
  END AS abnormal_to_normal_ratio
FROM counts;

 * sqlite:///UCI_database.db
Done.


abnormal_count,normal_count,abnormal_to_normal_ratio
152,151,1.0066225165562914


7. Number of Patients with Reversible Thalassemia:

- **PROMPT:** "Count the number of patients with reversible thalassemia detected by thallium stress testing."

In [29]:
%%sql 
-- 7 = reversable defect per dataset
SELECT
  COUNT(*) AS reversible_thal_count
FROM HEART_DISEASE_TABLE
WHERE thal = 7;  


 * sqlite:///UCI_database.db
Done.


reversible_thal_count
117


8. Average Age of Patients with Chest Pain:

- **PROMPT:** "Calculate the average age of patients who experienced chest pain during diagnosis."

In [30]:
%%sql 
-- Average age of patients who experienced chest pain during diagnosis
-- Chest pain types: 1=typical, 2=atypical, 3=non-anginal, 4=asymptomatic
-- We interpret "experienced chest pain" as cp in (1, 2, 3)
SELECT
  AVG(age) AS avg_age_chest_pain
FROM HEART_DISEASE_TABLE
WHERE age IS NOT NULL
  AND cp IN (1, 2, 3);

 * sqlite:///UCI_database.db
Done.


avg_age_chest_pain
53.276729559748425


9. Distribution of Patients by Number of Major Vessels:

- **PROMPT:** "Investigate the distribution of patients based on the number of major vessels colored by fluoroscopy (0-3)."

In [31]:
%%sql 
-- Distribution of patients by the number of major vessels colored by fluoroscopy (ca: 0-3)
-- ca values represent major vessels colored by fluoroscopy
-- NULL values are ignored; only 0-3 are counted
SELECT
  ca,
  CASE ca
    WHEN 0 THEN '0 vessels colored'
    WHEN 1 THEN '1 vessel colored'
    WHEN 2 THEN '2 vessels colored'
    WHEN 3 THEN '3 vessels colored'
    ELSE 'Unknown'
  END AS vessels_colored_label,
  COUNT(*) AS patient_count
FROM HEART_DISEASE_TABLE
WHERE ca IS NOT NULL
  AND ca BETWEEN 0 AND 3
GROUP BY ca
ORDER BY ca;

 * sqlite:///UCI_database.db
Done.


ca,vessels_colored_label,patient_count
0.0,0 vessels colored,176
1.0,1 vessel colored,65
2.0,2 vessels colored,38
3.0,3 vessels colored,20


In [32]:
con.close()